# Part I: Neural Networks using one-hot encoding

In [2]:
import numpy as np
import pandas as pd
from numpy import array
import keras
keras.__version__
from keras import models
from keras import layers
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras.layers.convolutional import Conv1D
from keras.layers.pooling import MaxPooling1D
from keras.layers import LSTM
from keras.layers import SimpleRNN
from keras.layers import BatchNormalization
from keras.utils import np_utils
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import StandardScaler
from keras.regularizers import L2
from keras.optimizers import Adam
from keras.layers import LeakyReLU
from sklearn.model_selection import train_test_split

Importing the data, eliminating unecessary columns, and excluding hours before 7 am.

In [3]:
data=pd.read_csv('C:\\Users\\jqotw\\joined.csv')

In [4]:
data=pd.DataFrame(data)
data=data.loc[data['hour']>6]
data.head()

,Unnamed: 0,year,month,day,hour,date,weekday,prop9E,propAA,propAS,...,daily_delays,TAVG.y,delays_yesterday,delays_past3d,hot3days,delays_past7d,hot7days,noschool,day_num,SET_NUM
3,4,2021,12,1,7,12/1/2021,4,0.0000,0.142857,0.0,...,84,45,0,0,0,0,0,0,1,3
4,5,2021,12,1,8,12/1/2021,4,0.0000,0.529412,0.0,...,84,45,0,0,0,0,0,0,1,3
5,6,2021,12,1,9,12/1/2021,4,0.0000,0.400000,0.0,...,84,45,0,0,0,0,0,0,1,3
6,7,2021,12,1,10,12/1/2021,4,0.0000,0.470588,0.0,...,84,45,0,0,0,0,0,0,1,3
7,8,2021,12,1,11,12/1/2021,4,0.0625,0.312500,0.0,...,84,45,0,0,0,0,0,0,1,3


In [501]:
data=data.iloc[:,[2,4,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,26,27,28,29,30,31,36,37,38,39,40,41,42,43,44,45,48,49,50,51,52,53]]
data.columns

Index(['month', 'hour', 'weekday', 'prop9E', 'propAA', 'propAS', 'propB6',
       'propDL', 'propF9', 'propNK', 'propOH', 'propOO', 'propUA', 'propWN',
       'propYV', 'propYX', 'regional', 'reg_carr', 'topdests', 'sched',
       'many_delays', 'AWND', 'PRCP', 'SNOW', 'SNWD', 'TAVG.x', 'WSF2', 'WSF5',
       'WT01', 'WT02', 'WT03', 'WT04', 'WT05', 'WT06', 'WT08', 'WT09',
       'delays_yesterday', 'delays_past3d', 'hot3days', 'delays_past7d',
       'hot7days', 'noschool'],
      dtype='object')

Scaling all continuous features

In [502]:

scaler=StandardScaler()

data['sched']=scaler.fit_transform(data[['sched']])
data['AWND']=scaler.fit_transform(data[['AWND']])
data['PRCP']=scaler.fit_transform(data[['PRCP']])
data['SNOW']=scaler.fit_transform(data[['SNOW']])
data['SNWD']=scaler.fit_transform(data[['SNWD']])
data['TAVG.x']=scaler.fit_transform(data[['TAVG.x']])
data['WSF2']=scaler.fit_transform(data[['WSF2']])
data['WSF5']=scaler.fit_transform(data[['WSF5']])
data['delays_past7d']=scaler.fit_transform(data[['delays_past7d']])
data['delays_yesterday']=scaler.fit_transform(data[['delays_yesterday']])
data['delays_past3d']=scaler.fit_transform(data[['delays_past3d']])
data['prop9E']=scaler.fit_transform(data[['prop9E']])
data['propUA']=scaler.fit_transform(data[['propUA']])
data['propF9']=scaler.fit_transform(data[['propF9']])
data['propYX']=scaler.fit_transform(data[['propYX']])
data['propOO']=scaler.fit_transform(data[['propOO']])
data['propDL']=scaler.fit_transform(data[['propDL']])
data['propAA']=scaler.fit_transform(data[['propAA']])
data['propWN']=scaler.fit_transform(data[['propWN']])
data['propNK']=scaler.fit_transform(data[['propNK']])
data['propAS']=scaler.fit_transform(data[['propAS']])
data['propOH']=scaler.fit_transform(data[['propOH']])
data['propB6']=scaler.fit_transform(data[['propB6']])
data['propYV']=scaler.fit_transform(data[['propYV']])
data['topdests']=scaler.fit_transform(data[['topdests']])
data['reg_carr']=scaler.fit_transform(data[['reg_carr']])
data['regional']=scaler.fit_transform(data[['regional']])

data.head()



,month,hour,weekday,prop9E,propAA,propAS,propB6,propDL,propF9,propNK,...,WT05,WT06,WT08,WT09,delays_yesterday,delays_past3d,hot3days,delays_past7d,hot7days,noschool
3,12,7,4,-0.182762,-1.237854,-0.275631,3.334616,-0.715998,1.323146,-0.591924,...,0,0,0,0,-1.648115,-1.894794,0,-2.093095,0,0
4,12,8,4,-0.182762,1.008444,-0.275631,-0.432823,0.210520,-0.409510,-0.591924,...,0,0,0,0,-1.648115,-1.894794,0,-2.093095,0,0
5,12,9,4,-0.182762,0.256423,-0.275631,-0.432823,-0.715998,-0.685160,-0.591924,...,0,0,0,0,-1.648115,-1.894794,0,-2.093095,0,0
6,12,10,4,-0.182762,0.666616,-0.275631,-0.432823,-0.715998,-0.409510,-0.591924,...,0,0,0,0,-1.648115,-1.894794,0,-2.093095,0,0
7,12,11,4,4.032065,-0.252046,-0.275631,-0.432823,0.268427,-0.392282,-0.164501,...,0,0,0,0,-1.648115,-1.894794,0,-2.093095,0,0


Converting month, hour, and weekday into one-hot encoded categories

In [503]:
data=pd.get_dummies(data,prefix=['month','hour','weekday'],columns=['month','hour','weekday'])
data.head()

,prop9E,propAA,propAS,propB6,propDL,propF9,propNK,propOH,propOO,propUA,...,hour_21,hour_22,hour_23,weekday_1,weekday_2,weekday_3,weekday_4,weekday_5,weekday_6,weekday_7
3,-0.182762,-1.237854,-0.275631,3.334616,-0.715998,1.323146,-0.591924,-0.161403,-0.258685,-0.607449,...,0,0,0,0,0,0,1,0,0,0
4,-0.182762,1.008444,-0.275631,-0.432823,0.210520,-0.409510,-0.591924,-0.699126,-0.258685,0.685409,...,0,0,0,0,0,0,1,0,0,0
5,-0.182762,0.256423,-0.275631,-0.432823,-0.715998,-0.685160,-0.591924,1.484029,1.588023,-0.607449,...,0,0,0,0,0,0,1,0,0,0
6,-0.182762,0.666616,-0.275631,-0.432823,-0.715998,-0.409510,-0.591924,0.806498,-0.258685,0.685409,...,0,0,0,0,0,0,1,0,0,0
7,4.032065,-0.252046,-0.275631,-0.432823,0.268427,-0.392282,-0.164501,0.124262,-0.258685,0.766212,...,0,0,0,0,0,0,1,0,0,0


Training and testing splits

In [504]:
train,test = train_test_split(data,test_size=.2)

In [505]:
Xtrain=train.loc[:,train.columns!='many_delays']

Xtest=test.loc[:,test.columns!='many_delays']


ytrain=train.loc[:,train.columns=='many_delays']
ytest=test.loc[:,test.columns=='many_delays']


FNN

In [556]:
fnn=Sequential()

fnn.add(Dense(512,kernel_regularizer=L2()))
fnn.add(BatchNormalization())
fnn.add(Activation(LeakyReLU()))
fnn.add(Dense(256,kernel_regularizer=L2()))
fnn.add(BatchNormalization())
fnn.add(Activation(LeakyReLU()))


fnn.add(Dense(1,activation='sigmoid'))
fnn.compile(optimizer='adam',
                loss='binary_crossentropy',
                metrics=['accuracy'])

In [591]:
fnn.fit(Xtrain,ytrain,epochs=20,batch_size=64)

Epoch 1/20
73/73 [==============================] - 0s 3ms/step - loss: 0.5261 - accuracy: 0.7588
Epoch 2/20
73/73 [==============================] - 0s 3ms/step - loss: 0.5130 - accuracy: 0.7796
Epoch 3/20
73/73 [==============================] - 0s 4ms/step - loss: 0.5052 - accuracy: 0.7753
Epoch 4/20
73/73 [==============================] - 0s 3ms/step - loss: 0.5055 - accuracy: 0.7757
Epoch 5/20
73/73 [==============================] - 0s 4ms/step - loss: 0.5260 - accuracy: 0.7627
Epoch 6/20
73/73 [==============================] - 0s 4ms/step - loss: 0.5184 - accuracy: 0.7647
Epoch 7/20
73/73 [==============================] - 0s 4ms/step - loss: 0.5122 - accuracy: 0.7681
Epoch 8/20
73/73 [==============================] - 0s 6ms/step - loss: 0.5165 - accuracy: 0.7686
Epoch 9/20
73/73 [==============================] - 0s 4ms/step - loss: 0.5056 - accuracy: 0.7790
Epoch 10/20
73/73 [==============================] - 0s 4ms/step - loss: 0.5140 - accuracy: 0.7753
Epoch 11/20
73/73 [

In [592]:
fnn.evaluate(Xtest,ytest)

37/37 [==============================] - 0s 2ms/step - loss: 0.7462 - accuracy: 0.6560


[0.7462417483329773, 0.6559792160987854]

1D CNN

In [607]:
cnn=Sequential()
cnn.add(Conv1D(filters=32,kernel_size=3,input_shape=(74,1)))
cnn.add(BatchNormalization())
cnn.add(Activation(LeakyReLU()))
cnn.add(Conv1D(filters=64,kernel_size=3))
cnn.add(BatchNormalization())
cnn.add(Activation(LeakyReLU()))
cnn.add(Conv1D(filters=128,kernel_size=3))
cnn.add(BatchNormalization())
cnn.add(Activation(LeakyReLU()))

cnn.add(MaxPooling1D(pool_size=(2)))



cnn.add(Flatten())
cnn.add(Dense(500))
cnn.add(BatchNormalization())
cnn.add(Activation(LeakyReLU()))

cnn.add(Dense(1,activation='sigmoid'))
cnn.compile(optimizer='adam',
                loss='binary_crossentropy',
                metrics=['accuracy'])
cnn.fit(Xtrain,ytrain,epochs=100,batch_size=32)

Epoch 1/100
145/145 [==============================] - 14s 31ms/step - loss: 0.9429 - accuracy: 0.6295
Epoch 2/100
145/145 [==============================] - 3s 22ms/step - loss: 0.6340 - accuracy: 0.6596
Epoch 3/100
145/145 [==============================] - 3s 24ms/step - loss: 0.6291 - accuracy: 0.6637
Epoch 4/100
145/145 [==============================] - 4s 26ms/step - loss: 0.6191 - accuracy: 0.6728
Epoch 5/100
145/145 [==============================] - 4s 27ms/step - loss: 0.6109 - accuracy: 0.6696
Epoch 6/100
145/145 [==============================] - 5s 32ms/step - loss: 0.6071 - accuracy: 0.6693
Epoch 7/100
145/145 [==============================] - 4s 29ms/step - loss: 0.5918 - accuracy: 0.6858
Epoch 8/100
145/145 [==============================] - 4s 28ms/step - loss: 0.5942 - accuracy: 0.6834
Epoch 9/100
145/145 [==============================] - 4s 28ms/step - loss: 0.5897 - accuracy: 0.6867
Epoch 10/100
145/145 [==============================] - 4s 27ms/step - loss: 0.58

KeyboardInterrupt: 

In [608]:
cnn.evaluate(Xtest,ytest)

37/37 [==============================] - 2s 14ms/step - loss: 0.7107 - accuracy: 0.6768


[0.7107442021369934, 0.6767764091491699]

Converting Data into recurrent tensors for RNN-structures (currently shown with 3 time-steps)

In [506]:
X=data.loc[:,data.columns!='many_delays']
y=data.loc[:,data.columns=='many_delays']
print(len(y))
x_y = list()
length = 3
n = 5769
data_array = array(data)
input_array = array(X)
output_array = array(y)
for i in range(0,n,length):

    xy = data_array[i:i+length]
    x_y.append(xy)
print(len(x_y))

xy_array = array(x_y)
print(xy_array.shape)



xy_array = xy_array.reshape((len(x_y), length, 75))
x = list()
length = 3
n = 5769


for i in range(0,n,length):

    x1 = input_array[i:i+length]
    x.append(x1)

print(len(x))

X_array = array(x)
print(X_array.shape)



X_array = X_array.reshape((len(x), length, 74))
y = list()
length = 3
n = 5769

for i in range(0,n,length):

    y1 = output_array[i+length-1]
    y.append(y1)
print(len(y))

y_array = array(y)
print(y_array.shape)



y_array = y_array.reshape((len(y),  1))

5769
1923
(1923, 3, 75)
1923
(1923, 3, 74)
1923
(1923, 1)


In [507]:
Train_inputs, Test_inputs, train_output, test_output = train_test_split(X_array, y_array, test_size=0.20, shuffle=True)


print(Train_inputs.shape)
print(Test_inputs.shape)
print(train_output.shape)
print(test_output.shape)


(1538, 3, 74)
(385, 3, 74)
(1538, 1)
(385, 1)


In [508]:
X_train2= Train_inputs.reshape(1538,3,74)
X_test2 = Test_inputs.reshape(385,3,74)
y_train2 = train_output.reshape(1538,1,1)
y_test2 = test_output.reshape(385,1,1)

RNN

In [517]:
rnn=Sequential()
rnn.add(SimpleRNN(200,activation='tanh',input_shape=(3,74),return_sequences=True))
rnn.add(SimpleRNN(100,activation='tanh',input_shape=(3,74),return_sequences=False))



rnn.add(Dense(32,activation='relu'))












rnn.add(Dense(1,activation='sigmoid'))

In [534]:
rnn.compile(optimizer='Adam',
                loss='binary_crossentropy',
                metrics=['accuracy'])
rnn.fit(X_train2,y_train2,epochs=4,batch_size=32)

Epoch 1/4
49/49 [==============================] - 6s 3ms/step - loss: 0.0280 - accuracy: 0.9928
Epoch 2/4
49/49 [==============================] - 0s 3ms/step - loss: 0.0185 - accuracy: 0.9967
Epoch 3/4
49/49 [==============================] - 0s 3ms/step - loss: 0.0179 - accuracy: 0.9954
Epoch 4/4
49/49 [==============================] - 0s 3ms/step - loss: 0.0333 - accuracy: 0.9935


In [535]:
loss,accuracy=rnn.evaluate(X_test2,y_test2)

13/13 [==============================] - 1s 2ms/step - loss: 1.6206 - accuracy: 0.6182


LSTM

In [624]:
lstm1=Sequential()
lstm1.add(LSTM(200,input_shape=(3,74),return_sequences=True))
lstm1.add(LSTM(100,input_shape=(3,74),return_sequences=False))

lstm1.add(Dense(32,activation='relu'))



lstm1.add(Dense(1,activation='sigmoid'))


In [625]:
lstm1.compile(optimizer='Adam',
                loss='binary_crossentropy',
                metrics=['accuracy'])
lstm1.fit(X_train2,y_train2,epochs=10,batch_size=32)

Epoch 1/10
49/49 [==============================] - 11s 9ms/step - loss: 0.6681 - accuracy: 0.5988
Epoch 2/10
49/49 [==============================] - 0s 8ms/step - loss: 0.6274 - accuracy: 0.6489
Epoch 3/10
49/49 [==============================] - 0s 8ms/step - loss: 0.6141 - accuracy: 0.6697
Epoch 4/10
49/49 [==============================] - 0s 9ms/step - loss: 0.5951 - accuracy: 0.6762
Epoch 5/10
49/49 [==============================] - 1s 12ms/step - loss: 0.5754 - accuracy: 0.6938
Epoch 6/10
49/49 [==============================] - 1s 10ms/step - loss: 0.5590 - accuracy: 0.7113
Epoch 7/10
49/49 [==============================] - 0s 8ms/step - loss: 0.5407 - accuracy: 0.7191
Epoch 8/10
49/49 [==============================] - 0s 9ms/step - loss: 0.5233 - accuracy: 0.7347
Epoch 9/10
49/49 [==============================] - 1s 14ms/step - loss: 0.4908 - accuracy: 0.7555
Epoch 10/10
49/49 [==============================] - 0s 10ms/step - loss: 0.4573 - accuracy: 0.7822


In [627]:
loss,accuracy=lstm1.evaluate(X_test2,y_test2)

13/13 [==============================] - 0s 3ms/step - loss: 0.7061 - accuracy: 0.6779


# Categorical Embedding Section

In [5]:
import categorical_embedder as ce

Re-loading and scaling data freshly

In [6]:
data=pd.read_csv('C:\\Users\\jqotw\\joined.csv')
data=pd.DataFrame(data)
data.head()

,Unnamed: 0,year,month,day,hour,date,weekday,prop9E,propAA,propAS,...,daily_delays,TAVG.y,delays_yesterday,delays_past3d,hot3days,delays_past7d,hot7days,noschool,day_num,SET_NUM
0,1,2021,12,1,0,12/1/2021,4,0.0,0.000000,0.0,...,84,45,0,0,0,0,0,0,1,3
1,2,2021,12,1,5,12/1/2021,4,0.0,0.600000,0.0,...,84,45,0,0,0,0,0,0,1,3
2,3,2021,12,1,6,12/1/2021,4,0.0,0.235294,0.0,...,84,45,0,0,0,0,0,0,1,3
3,4,2021,12,1,7,12/1/2021,4,0.0,0.142857,0.0,...,84,45,0,0,0,0,0,0,1,3
4,5,2021,12,1,8,12/1/2021,4,0.0,0.529412,0.0,...,84,45,0,0,0,0,0,0,1,3


In [7]:
data=data.loc[data['hour']>6]
data=data.iloc[:,[2,4,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,26,27,28,29,30,31,36,37,38,39,40,41,42,43,44,45,48,49,50,51,52,53]]

len(data)
scaler=StandardScaler()

data['sched']=scaler.fit_transform(data[['sched']])
data['AWND']=scaler.fit_transform(data[['AWND']])
data['PRCP']=scaler.fit_transform(data[['PRCP']])
data['SNOW']=scaler.fit_transform(data[['SNOW']])
data['SNWD']=scaler.fit_transform(data[['SNWD']])
data['TAVG.x']=scaler.fit_transform(data[['TAVG.x']])
data['WSF2']=scaler.fit_transform(data[['WSF2']])
data['WSF5']=scaler.fit_transform(data[['WSF5']])
data['delays_past7d']=scaler.fit_transform(data[['delays_past7d']])
data['delays_yesterday']=scaler.fit_transform(data[['delays_yesterday']])
data['delays_past3d']=scaler.fit_transform(data[['delays_past3d']])
data['prop9E']=scaler.fit_transform(data[['prop9E']])
data['propUA']=scaler.fit_transform(data[['propUA']])
data['propF9']=scaler.fit_transform(data[['propF9']])
data['propYX']=scaler.fit_transform(data[['propYX']])
data['propOO']=scaler.fit_transform(data[['propOO']])
data['propDL']=scaler.fit_transform(data[['propDL']])
data['propAA']=scaler.fit_transform(data[['propAA']])
data['propWN']=scaler.fit_transform(data[['propWN']])
data['propNK']=scaler.fit_transform(data[['propNK']])
data['propAS']=scaler.fit_transform(data[['propAS']])
data['propOH']=scaler.fit_transform(data[['propOH']])
data['propB6']=scaler.fit_transform(data[['propB6']])
data['propYV']=scaler.fit_transform(data[['propYV']])
data['topdests']=scaler.fit_transform(data[['topdests']])
data['reg_carr']=scaler.fit_transform(data[['reg_carr']])
data['regional']=scaler.fit_transform(data[['regional']])

data.head()

,month,hour,weekday,prop9E,propAA,propAS,propB6,propDL,propF9,propNK,...,WT05,WT06,WT08,WT09,delays_yesterday,delays_past3d,hot3days,delays_past7d,hot7days,noschool
3,12,7,4,-0.182762,-1.237854,-0.275631,3.334616,-0.715998,1.323146,-0.591924,...,0,0,0,0,-1.648115,-1.894794,0,-2.093095,0,0
4,12,8,4,-0.182762,1.008444,-0.275631,-0.432823,0.210520,-0.409510,-0.591924,...,0,0,0,0,-1.648115,-1.894794,0,-2.093095,0,0
5,12,9,4,-0.182762,0.256423,-0.275631,-0.432823,-0.715998,-0.685160,-0.591924,...,0,0,0,0,-1.648115,-1.894794,0,-2.093095,0,0
6,12,10,4,-0.182762,0.666616,-0.275631,-0.432823,-0.715998,-0.409510,-0.591924,...,0,0,0,0,-1.648115,-1.894794,0,-2.093095,0,0
7,12,11,4,4.032065,-0.252046,-0.275631,-0.432823,0.268427,-0.392282,-0.164501,...,0,0,0,0,-1.648115,-1.894794,0,-2.093095,0,0


In [8]:
X = data.drop(['many_delays'], axis=1)

y = data['many_delays']

Checking how data-types are parsed and converting categorical values to string so embedder recognizes them as categorical variables

In [301]:
print(X.dtypes)


month                 int64
hour                  int64
weekday               int64
prop9E              float64
propAA              float64
propAS              float64
propB6              float64
propDL              float64
propF9              float64
propNK              float64
propOH              float64
propOO              float64
propUA              float64
propWN              float64
propYV              float64
propYX              float64
regional            float64
reg_carr            float64
topdests            float64
sched               float64
AWND                float64
PRCP                float64
SNOW                float64
SNWD                float64
TAVG.x              float64
WSF2                float64
WSF5                float64
WT01                  int64
WT02                  int64
WT03                  int64
WT04                  int64
WT05                  int64
WT06                  int64
WT08                  int64
WT09                  int64
delays_yesterday    

In [9]:
X['hot3days']=X['hot3days'].astype(str)
X['hot7days']=X['hot7days'].astype(str)
X['month']=X['month'].astype(str)
X['hour']=X['hour'].astype(str)
X['weekday']=X['weekday'].astype(str)
X['noschool']=X['noschool'].astype(str)
X['month']=X['month'].astype(str)
X['WT09']=X['WT09'].astype(str)
X['WT08']=X['WT08'].astype(str)
X['WT06']=X['WT06'].astype(str)
X['WT05']=X['WT05'].astype(str)
X['WT04']=X['WT04'].astype(str)
X['WT03']=X['WT03'].astype(str)
X['WT02']=X['WT02'].astype(str)
X['WT01']=X['WT01'].astype(str)

Creating the embeddings

In [10]:
embedding_info = ce.get_embedding_info(X)
embedding_info

{'month': (12, 6),
 'hour': (17, 9),
 'weekday': (7, 4),
 'WT01': (2, 1),
 'WT02': (2, 1),
 'WT03': (2, 1),
 'WT04': (2, 1),
 'WT05': (2, 1),
 'WT06': (2, 1),
 'WT08': (2, 1),
 'WT09': (2, 1),
 'hot3days': (2, 1),
 'hot7days': (2, 1),
 'noschool': (2, 1)}

In [11]:
X_encoded,encoders = ce.get_label_encoded_data(X)
X_encoded.head()

,month,hour,weekday,prop9E,propAA,propAS,propB6,propDL,propF9,propNK,...,WT05,WT06,WT08,WT09,delays_yesterday,delays_past3d,hot3days,delays_past7d,hot7days,noschool
3,3,14,3,-0.182762,-1.237854,-0.275631,3.334616,-0.715998,1.323146,-0.591924,...,0,0,0,0,-1.648115,-1.894794,0,-2.093095,0,0
4,3,15,3,-0.182762,1.008444,-0.275631,-0.432823,0.210520,-0.409510,-0.591924,...,0,0,0,0,-1.648115,-1.894794,0,-2.093095,0,0
5,3,16,3,-0.182762,0.256423,-0.275631,-0.432823,-0.715998,-0.685160,-0.591924,...,0,0,0,0,-1.648115,-1.894794,0,-2.093095,0,0
6,3,0,3,-0.182762,0.666616,-0.275631,-0.432823,-0.715998,-0.409510,-0.591924,...,0,0,0,0,-1.648115,-1.894794,0,-2.093095,0,0
7,3,1,3,4.032065,-0.252046,-0.275631,-0.432823,0.268427,-0.392282,-0.164501,...,0,0,0,0,-1.648115,-1.894794,0,-2.093095,0,0


In [12]:
X_train, X_test, y_train, y_test = train_test_split(X_encoded,y)

# ce.get_embeddings trains NN, extracts embeddings and return a dictionary containing the embeddings
embeddings = ce.get_embeddings(X_train, y_train, categorical_embedding_info=embedding_info, 
                            is_classification=True, epochs=100,batch_size=256)

In [13]:
X = ce.fit_transform(X, embeddings=embeddings, encoders=encoders, drop_categorical_vars=True)
X.head()

  0%|          | 0/14 [00:00<?, ?it/s]

,prop9E,propAA,propAS,propB6,propDL,propF9,propNK,propOH,propOO,propUA,...,WT02_embedding_0,WT03_embedding_0,WT04_embedding_0,WT05_embedding_0,WT06_embedding_0,WT08_embedding_0,WT09_embedding_0,hot3days_embedding_0,hot7days_embedding_0,noschool_embedding_0
3,-0.182762,-1.237854,-0.275631,3.334616,-0.715998,1.323146,-0.591924,-0.161403,-0.258685,-0.607449,...,-0.036755,0.004777,0.012467,-0.065335,0.038951,0.03344,-0.054455,0.095918,-0.042475,0.10279
4,-0.182762,1.008444,-0.275631,-0.432823,0.210520,-0.409510,-0.591924,-0.699126,-0.258685,0.685409,...,-0.036755,0.004777,0.012467,-0.065335,0.038951,0.03344,-0.054455,0.095918,-0.042475,0.10279
5,-0.182762,0.256423,-0.275631,-0.432823,-0.715998,-0.685160,-0.591924,1.484029,1.588023,-0.607449,...,-0.036755,0.004777,0.012467,-0.065335,0.038951,0.03344,-0.054455,0.095918,-0.042475,0.10279
6,-0.182762,0.666616,-0.275631,-0.432823,-0.715998,-0.409510,-0.591924,0.806498,-0.258685,0.685409,...,-0.036755,0.004777,0.012467,-0.065335,0.038951,0.03344,-0.054455,0.095918,-0.042475,0.10279
7,4.032065,-0.252046,-0.275631,-0.432823,0.268427,-0.392282,-0.164501,0.124262,-0.258685,0.766212,...,-0.036755,0.004777,0.012467,-0.065335,0.038951,0.03344,-0.054455,0.095918,-0.042475,0.10279


Running same experiment as before

In [14]:

Xtrain, Xtest, ytrain, ytest = train_test_split(X,y,test_size=.2)

In [638]:
fnn=Sequential()

fnn.add(Dense(512,kernel_regularizer=L2()))
fnn.add(BatchNormalization())
fnn.add(Activation(LeakyReLU()))
fnn.add(Dense(256,kernel_regularizer=L2()))
fnn.add(BatchNormalization())
fnn.add(Activation(LeakyReLU()))


fnn.add(Dense(1,activation='sigmoid'))
fnn.compile(optimizer='adam',
                loss='binary_crossentropy',
                metrics=['accuracy'])

In [643]:
fnn.fit(Xtrain,ytrain,epochs=100,batch_size=64)

Epoch 1/100
73/73 [==============================] - 0s 3ms/step - loss: 0.5564 - accuracy: 0.7333
Epoch 2/100
73/73 [==============================] - 0s 3ms/step - loss: 0.5514 - accuracy: 0.7257
Epoch 3/100
73/73 [==============================] - 0s 3ms/step - loss: 0.5472 - accuracy: 0.7354
Epoch 4/100
73/73 [==============================] - 0s 3ms/step - loss: 0.5428 - accuracy: 0.7428
Epoch 5/100
73/73 [==============================] - 0s 3ms/step - loss: 0.5544 - accuracy: 0.7250
Epoch 6/100
73/73 [==============================] - 0s 3ms/step - loss: 0.5533 - accuracy: 0.7341
Epoch 7/100
73/73 [==============================] - 0s 3ms/step - loss: 0.5518 - accuracy: 0.7322
Epoch 8/100
73/73 [==============================] - 0s 3ms/step - loss: 0.5538 - accuracy: 0.7348
Epoch 9/100
73/73 [==============================] - 0s 3ms/step - loss: 0.5530 - accuracy: 0.7311
Epoch 10/100
73/73 [==============================] - 0s 3ms/step - loss: 0.5582 - accuracy: 0.7285
Epoch 11/

In [644]:
fnn.evaluate(Xtest,ytest)

37/37 [==============================] - 0s 2ms/step - loss: 0.6789 - accuracy: 0.6629


[0.6789455413818359, 0.662911593914032]

In [46]:
cnn=Sequential()
cnn.add(Conv1D(filters=32,kernel_size=3,input_shape=(57,1)))
cnn.add(BatchNormalization())
cnn.add(Activation(LeakyReLU()))
cnn.add(Conv1D(filters=64,kernel_size=3))
cnn.add(BatchNormalization())
cnn.add(Activation(LeakyReLU()))
cnn.add(Conv1D(filters=128,kernel_size=3))
cnn.add(BatchNormalization())
cnn.add(Activation(LeakyReLU()))

cnn.add(MaxPooling1D(pool_size=(2)))



cnn.add(Flatten())
cnn.add(Dense(500))
cnn.add(BatchNormalization())
cnn.add(Activation(LeakyReLU()))

cnn.add(Dense(1,activation='sigmoid'))
cnn.compile(optimizer='adam',
                loss='binary_crossentropy',
                metrics=['accuracy'])
cnn.fit(Xtrain,ytrain,epochs=20,batch_size=32)

Epoch 1/20
145/145 [==============================] - 4s 17ms/step - loss: 0.7908 - accuracy: 0.6212
Epoch 2/20
145/145 [==============================] - 2s 17ms/step - loss: 0.6269 - accuracy: 0.6518
Epoch 3/20
145/145 [==============================] - 3s 19ms/step - loss: 0.6113 - accuracy: 0.6663
Epoch 4/20
145/145 [==============================] - 3s 20ms/step - loss: 0.6029 - accuracy: 0.6767
Epoch 5/20
145/145 [==============================] - 3s 20ms/step - loss: 0.5901 - accuracy: 0.6858
Epoch 6/20
145/145 [==============================] - 3s 20ms/step - loss: 0.5862 - accuracy: 0.6830
Epoch 7/20
145/145 [==============================] - 3s 20ms/step - loss: 0.5771 - accuracy: 0.6949
Epoch 8/20
145/145 [==============================] - 3s 18ms/step - loss: 0.5787 - accuracy: 0.6897
Epoch 9/20
145/145 [==============================] - 3s 20ms/step - loss: 0.5706 - accuracy: 0.7010
Epoch 10/20
145/145 [==============================] - 3s 20ms/step - loss: 0.5632 - accura

In [47]:
cnn.evaluate(Xtest,ytest)

37/37 [==============================] - 1s 6ms/step - loss: 0.6619 - accuracy: 0.6534


[0.6618863940238953, 0.6533795595169067]

In [48]:
input_array=array(X)
output_array=array(y)

In [49]:

x = list()
length = 3
n = 5769
output_samples = list()

for i in range(0,n,length):

    x1 = input_array[i:i+length]
    x.append(x1)

print(len(x))

X_array = array(x)
print(X_array.shape)



X_array = X_array.reshape((len(x), length, 57))
y = list()
length = 3
n = 5769

for i in range(0,n,length):

    y1 = output_array[i+length-1]
    y.append(y1)
print(len(y))

y_array = array(y)
print(y_array.shape)



y_array = y_array.reshape((len(y),  1))

1923
(1923, 3, 57)
1923
(1923,)


In [50]:
Train_inputs, Test_inputs, train_output, test_output = train_test_split(X_array, y_array, test_size=0.20, shuffle=True)


print(Train_inputs.shape)
print(Test_inputs.shape)
print(train_output.shape)
print(test_output.shape)


(1538, 3, 57)
(385, 3, 57)
(1538, 1)
(385, 1)


In [51]:
X_train2= Train_inputs.reshape(1538,3,57)
X_test2 = Test_inputs.reshape(385,3,57)
y_train2 = train_output.reshape(1538,1,1)
y_test2 = test_output.reshape(385,1,1)

In [75]:
lstm1=Sequential()
lstm1.add(LSTM(200,input_shape=(3,57),return_sequences=True))
lstm1.add(LSTM(100,input_shape=(3,57),return_sequences=False))

lstm1.add(Dense(32,activation='relu'))



lstm1.add(Dense(1,activation='sigmoid'))



In [78]:
lstm1.compile(optimizer='Adam',
                loss='binary_crossentropy',
                metrics=['accuracy'])
lstm1.fit(X_train2,y_train2,epochs=10,batch_size=32)

Epoch 1/10
49/49 [==============================] - 6s 7ms/step - loss: 0.4816 - accuracy: 0.7685
Epoch 2/10
49/49 [==============================] - 0s 6ms/step - loss: 0.4481 - accuracy: 0.7861
Epoch 3/10
49/49 [==============================] - 0s 7ms/step - loss: 0.4334 - accuracy: 0.7893
Epoch 4/10
49/49 [==============================] - 0s 7ms/step - loss: 0.4109 - accuracy: 0.8134
Epoch 5/10
49/49 [==============================] - 0s 6ms/step - loss: 0.3892 - accuracy: 0.8205
Epoch 6/10
49/49 [==============================] - 0s 6ms/step - loss: 0.3620 - accuracy: 0.8264
Epoch 7/10
49/49 [==============================] - 0s 6ms/step - loss: 0.3545 - accuracy: 0.8349
Epoch 8/10
49/49 [==============================] - 0s 6ms/step - loss: 0.3163 - accuracy: 0.8544
Epoch 9/10
49/49 [==============================] - 0s 6ms/step - loss: 0.3485 - accuracy: 0.8446
Epoch 10/10
49/49 [==============================] - 0s 6ms/step - loss: 0.2737 - accuracy: 0.8758


In [79]:
loss,accuracy=lstm1.evaluate(X_test2,y_test2)

13/13 [==============================] - 1s 5ms/step - loss: 1.0590 - accuracy: 0.6182


In [80]:
rnn=Sequential()
rnn.add(SimpleRNN(200,activation='tanh',input_shape=(3,57),return_sequences=True))
rnn.add(SimpleRNN(100,activation='tanh',input_shape=(3,57),return_sequences=False))



rnn.add(Dense(32,activation ='relu'))












rnn.add(Dense(1,activation='sigmoid'))

In [85]:
rnn.compile(optimizer='Adam',
                loss='binary_crossentropy',
                metrics=['accuracy'])
rnn.fit(X_train2,y_train2,epochs=3,batch_size=32)

Epoch 1/3
49/49 [==============================] - 2s 3ms/step - loss: 0.4290 - accuracy: 0.8114
Epoch 2/3
49/49 [==============================] - 0s 2ms/step - loss: 0.3685 - accuracy: 0.8472
Epoch 3/3
49/49 [==============================] - 0s 3ms/step - loss: 0.3268 - accuracy: 0.8700


In [86]:
loss,accuracy=rnn.evaluate(X_test2,y_test2)

13/13 [==============================] - 8s 9ms/step - loss: 0.7814 - accuracy: 0.6130
